In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
eigval = pd.read_csv('../build/debug/eigval.csv')
eigvec = pd.read_csv('../build/debug/eigvec.csv', header=None).values

In [ ]:
plt.plot(eigval.rho, eigval.result)

In [ ]:
plt.plot(eigvec[0,:19])

In [ ]:
plt.imshow(eigvec, interpolation='nearest', norm=LogNorm())